In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedKFold



In [2]:
def kfold(df = None, time_stamp = 20):
    for label in [0, 1, 2, 3, 4]:
        df_label = df[df['label'] == label]
        
        for i in range(len(df_label)//time_stamp):
            if i == len(df_label)//time_stamp - 1:
                df_stamp = df_label[i*time_stamp:]
            else:
                df_stamp = df_label[i*time_stamp: i*time_stamp + 20]
            
            skf = StratifiedKFold(n_splits=5, shuffle=True)
            X = df_stamp['image_id']
            Y = df_stamp['label']
            fold = 0
            for train_index, test_index in skf.split(X, Y):
                fold += 1

                X_test = X.iloc[test_index]
                print(df.image_id.isin(X_test))
                df.loc[df.image_id.isin(X_test), 'fold'] = fold
    return df 

In [44]:
df_new = pd.read_csv('new.csv')
X = df_new['image_id']
Y = df_new['label']
df_new['fold'] = 0

df_new = df_new.sort_values(by=['image_id'])

In [45]:
df_new = kfold(df_new)

0        False
1        False
2        False
3        False
4        False
         ...  
21390    False
21391    False
21392    False
21393    False
21394    False
Name: image_id, Length: 21395, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
21390    False
21391    False
21392    False
21393    False
21394    False
Name: image_id, Length: 21395, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
21390    False
21391    False
21392    False
21393    False
21394    False
Name: image_id, Length: 21395, dtype: bool
0         True
1        False
2        False
3        False
4        False
         ...  
21390    False
21391    False
21392    False
21393    False
21394    False
Name: image_id, Length: 21395, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
21390    False
21391    False
21392    False
21393    False
21394    False
Nam

KeyboardInterrupt: 

In [5]:
df_new[df_new['label'] == 0]['fold'][:].value_counts()

1    218
5    217
4    217
3    217
2    217
Name: fold, dtype: int64

In [8]:
df_old = pd.read_csv('old.csv')
X = df_old['image_id']
Y = df_old['label']
df_old['fold'] = 0

df_old = df_old.sort_values(by=['image_id'])

In [9]:
df_old = kfold(df_old)

In [10]:
frames = [df_new, df_old]
df_all = pd.concat(frames)
# df_all.to_csv('new_mix_v3.csv', index = False)

In [11]:
# df = pd.read_csv('new_mix_v3.csv')
# df = pd.read_csv('/home/hana/sonnh/kaggle-cassava/dataset/train_mix/new_mix_1234.csv')
df = pd.read_csv('/home/hana/sonnh/kaggle-cassava/dataset/train_mix/new_mix_v3.csv')
df.head()

,image_id,label,fold
0,1000015157.jpg,0,1
1,1000201771.jpg,3,2
2,100042118.jpg,1,3
3,1000723321.jpg,1,4
4,1000812911.jpg,3,3


In [10]:
df[df['label'] == 4]['fold'][:100].value_counts()

5    20
4    20
3    20
2    20
1    20
Name: fold, dtype: int64

# fuse

In [5]:
fold_fuse=[1,3]

In [6]:
import copy

def fuse_fold(df = None, fold_fuse = [], time_stamp = 20):
    df_res = copy.deepcopy(df)
    for fold in fold_fuse:
        df_fold = df[df['fold'] == fold]
        for label in [0, 1, 2, 3, 4]:
            df_label = df_fold[df_fold['label'] == label]

            for i in range(len(df_label)//time_stamp):
                if i == len(df_label)//time_stamp - 1:
                    df_stamp = df_label[i*time_stamp:]
                else:
                    df_stamp = df_label[i*time_stamp: i*time_stamp + 20]

                skf = StratifiedKFold(n_splits=len(fold_fuse), shuffle=True)
                X = df_stamp['image_id']
                Y = df_stamp['label']
                fold_count = 0
                for train_index, test_index in skf.split(X, Y):
                    
                    X_test = X.iloc[test_index]
                    df_res.loc[df_res.image_id.isin(X_test), 'fold'] = fold_fuse[fold_count]
                    fold_count += 1
    return df_res 

In [7]:
df = pd.read_csv('/home/hana/sonnh/kaggle-cassava/dataset/train_mix/new_mix_v3_fuse1_fold34.csv')
df_new = df[:21395]
df_new = df_new.sort_values(by=['image_id'])
df_old = df[21395:]
df_old = df_old.sort_values(by=['image_id'])


# print(df_new.head())
# print(df_old.head())
df_new = fuse_fold(df_new, fold_fuse)
df_old = fuse_fold(df_old, fold_fuse)
print(df_new.head())
print(df_old.head())

         image_id  label  fold
0  1000015157.jpg      0     3
1  1000201771.jpg      3     2
2   100042118.jpg      1     4
3  1000723321.jpg      1     3
4  1000812911.jpg      3     3
                image_id  label  fold
21395    train-cbb-0.jpg      0     2
21396    train-cbb-1.jpg      0     5
21397   train-cbb-10.jpg      0     4
21398  train-cbb-101.jpg      0     2
21399  train-cbb-102.jpg      0     2


In [8]:
frames = [df_new, df_old]
df_all = pd.concat(frames)
# df_all.to_csv('new_mix_v3_fuse2_fold13.csv', index = False)

# recheck

In [77]:
df_new[df_new['label'] == 2]['fold'][:200].value_counts()

5    40
4    40
3    40
2    40
1    40
Name: fold, dtype: int64

In [9]:
df = pd.read_csv('/home/hana/sonnh/kaggle-cassava/dataset/train_mix/new_mix_v3_fuse2_fold13.csv')
df_new = df[:21395]
df_new[df_new['label'] == 4]['fold'][:100].value_counts()

5    20
4    20
3    20
2    20
1    20
Name: fold, dtype: int64